In [600]:
function Make_Profile(k, t, motifs, pseudo_counts)

    count_dic = Dict('A' => [], 'G'=> [], 'T'=> [], 'C'=> [])

    for j = 1:k
        A_num = 0
        G_num = 0
        T_num = 0
        C_num = 0
        for i = 1:t
            if motifs[i][j] == 'A'
                A_num += 1
            elseif motifs[i][j] == 'G'
                G_num += 1
            elseif motifs[i][j] == 'T'
                T_num += 1
            elseif motifs[i][j] == 'C'
                C_num += 1
            end
        end
        push!(count_dic['A'], A_num) 
        push!(count_dic['G'], G_num)
        push!(count_dic['T'], T_num)
        push!(count_dic['C'], C_num)
    end

    #print(count_dic)

    if pseudo_counts == false
        for letter in ['A', 'G', 'T', 'C']
            for i in 1:length(count_dic[letter])
                count_dic[letter][i] /= t
            end
        end
    
    else
        
        for letter in ['A', 'G', 'T', 'C']
            for i in 1:length(count_dic[letter])
                count_dic[letter][i] += 1
                count_dic[letter][i]/= (t + 4)
            end
        end        
        
        
    end
    #print(count_dic)
    
     return count_dic
end

Make_Profile (generic function with 1 method)

In [601]:
function Get_Kmer(k, profile, DNA, deterministic)
    l = length(DNA[1])
    pro_list = []
    candidate_list = []
    max_list = []
    for i in 1:length(DNA)
        temp = []
        temp2 = []
        temp3 = ""
        max_pro = 0
        for j in 1:l - k + 1
            candidate = DNA[i][j:j+k-1]
            push!(temp2, candidate)
            pro = 1
            for char_idx in 1:length(candidate)
                pro = pro * profile[candidate[char_idx]][char_idx]
            end
            push!(temp, pro)
            if pro >= max_pro
                max_pro = pro
                temp3 = candidate
            end
            #println(pro)
            #print("   ")
        end
        push!(pro_list, temp)
        push!(candidate_list, temp2)
        push!(max_list, temp3)
    end
    
    
    if deterministic == true
        return max_list

    else
        res = []
        for i in 1:length(candidate_list)
            items = candidate_list[i]
            weights = pro_list[i]
            weights = convert(Array{Float64,1},weights)
            #print(weights)
            push!(res, sample(items, Weights(weights)))
        end
        return  res
    end

end

Get_Kmer (generic function with 1 method)

In [602]:
function getRandomMotif(DNA, k, t)
    init_motif = []
    l = length(DNA[1])
    #print(l)
    for i = 1:t
        random_s = rand(1:l-k+1)
        push!(init_motif, DNA[i][random_s:random_s + k - 1])
    end
    return init_motif
end

getRandomMotif (generic function with 1 method)

In [603]:
function motifScore(k, t, motifs)
    motif_score = 0
    #println(motifs)
    profile = Make_Profile(k, t, motifs, true)
    #println("profile:",profile)
    char_list = []
    pro_max_char = ""
    for j = 1:length(motifs[1])
        pro_max = 0
        for i = ['A', 'G', 'T', 'C']
            if profile[i][j] >= pro_max
                pro_max = profile[i][j]
                pro_max_char = i
            end
        end
        #println("pro_max:",pro_max)
        push!(char_list, pro_max_char)
    end
    
    #println("char_list:",char_list)
    for i = 1:length(motifs)
        #println("i:",i)
        for j = 1:length(motifs[1])
            #println("j:",j)
            if motifs[i][j] != char_list[j]
                motif_score += 1
            end
        end
    end
    
    return motif_score
end

motifScore (generic function with 2 methods)

In [645]:
function GIBBSSAMPLER(DNA, k, t, N)
    # randomly select k-mers Motifs = (Motif1, …, Motift) in each string from DNA
    motifs = getRandomMotif(DNA, k, t)
    #print(motifs)
    bestMotifs = motifs
    
    for j = 1:N
        #print(j)
        i = rand(1:t)
        cur_motif = []
        #println("i:",i)
        #println(motifs)
        for idx = 1:length(motifs)
            #print("idx:",idx)
            if idx != i
                push!(cur_motif, motifs[idx])
            end
        end
        #println("cur:", cur_motif)
        profile = Make_Profile(k, t-1, cur_motif, true)
        #print(profile)
        Kmer = Get_Kmer(k, profile, DNA, false)
        sub_motif_i = Kmer[i]
        #println("sub_motif_i:",sub_motif_i)
        motifs[i] = sub_motif_i
        #println("motifs:",motifs)
        motif_score = motifScore(k, t, motifs)
        #println("motif_score:",motif_score)
        best_score = motifScore(k, t, bestMotifs)
        if motif_score < best_score
            bestMotifs = motifs
        end
    end
    return bestMotifs
    
end

GIBBSSAMPLER (generic function with 1 method)

In [646]:
function readfile(fname)
    k = 0
    t = 0
    n = 0
    str = []
    open(fname) do file
        k,t,n = parse(Int, readuntil(file," ")), parse(Int, readuntil(file," ")),parse(Int, readuntil(file,"\n"))
        for ln in eachline(file)
            chars = []
            
            for i in 1:length(ln)
                push!(chars, uppercase(ln[i]))
            end
            
            push!(str,chars)
            
        end
    end
    
    return k, t, n, str
end

readfile (generic function with 2 methods)

In [716]:
using Pkg
Pkg.add("StatsBase")
using StatsBase



for fname in ["test1.txt","test2.txt","test3.txt"]
    println("fname:",fname)
    scores = []
    for i = 1:10
        k,t,n,DNA = readfile(fname)

        #print(k,t,n,DNA)

        bestMotifs = GIBBSSAMPLER(DNA, k, t, n)

        #println(bestMotifs)
        score = motifScore(k,t,bestMotifs)
        #println("score:",score)
        push!(scores,score)
    end
    println("scores:", scores)
    println("ave_score:",sum(scores)/length(scores))
end


 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
fname:test1.txt
scores:Any[14, 17, 17, 19, 15, 14, 15, 15, 15, 17]
ave_score:15.8
fname:test2.txt
scores:Any[7, 12, 5, 9, 18, 9, 17, 12, 5, 7]
ave_score:10.1
fname:test3.txt
scores:Any[8, 16, 8, 12, 15, 8, 16, 8, 12, 8]
ave_score:11.1


The default pseudo_counts is set to true, and the deterministic is set to false.
I run the GIBBSSAMPLER 10 time, and get the average score to see the performance. The score of sample in the homework is around 14 which is pretty far from the answer 9. I am not sure if it is because of the number of n is not big enough. And I also collect the data from http://jaspar.genereg.net/downloads/ to get other samples.Also, n is set to 300.
(The score is set to be the lower the better)
Average score of test2.txt is 8.9, and correct answer is 5
Average score of test3.txt is 12.7, and correct answer is 8
After observation, the gibsamppleing method can "sometime" get the best answer, about 0.2 chance that it can produce the best answer, but others could be pretty bad score, so it leads to not really good average score.